In [91]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split, SequentialSampler, BatchSampler, RandomSampler
import torch.optim as optim

In [ ]:
def greedy_dss(X_train, X_val, theta, eta, k, r, lambd, R, sel, loss, loss_grad):
    '''
    Attributes:
    -----
    
    X_train: ndarray
        Training set (U)
    X_val: ndarray
        Validation set (V)
    theta: ndarray
        Vector of current parameters
    eta: int
        Learning rate
    k: ??
        Budget
    r: int
        Number of times to do Taylor-series approximation
    lambd: int
        Regularization coefficient
    R: function
        Regularization function
    sel: String
        Selection method type, one of {'naive_greedy', 'stochastic_greedy'}
    loss: function
        Function which accepts training subsample and a vector of parameters and returns value of LLT
    loss_grad: function
        Function which accepts training subsample and a vector of parameters and returns gradient of LLT
    '''
    
    S = None
    U = X_train
    eps = 1e-5
    
    for i in range(r):
        if sel == 'naive_greedy':
            V = U
        else if sel == 'stochastic_greedy':
            inds = np.random.choice(np.arange(U.shape[0]), 
                                    size = U.shape[0] / (r * np.log(eps)), 
                                    replace = False)
            V = U[inds, :]
    
        theta = theta + eta * loss_grad(V, theta)
        G = G()

In [277]:
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        output = self.fc2(x)
        return output

In [343]:
from queue import PriorityQueue

class SetFunctionLoader_2(object):
    def __init__(self, trainset, x_val, y_val, model, loss_criterion,
                 loss_nored, eta, device, num_classes, batch_size):
        self.trainset = trainset  # assume its a sequential loader.
        self.x_val = x_val
        self.y_val = y_val
        self.model = model
        self.loss = loss_criterion  # Make sure it has reduction='none' instead of default
        self.loss_nored = loss_nored
        self.eta = eta  # step size for the one step gradient update
        # self.opt = optimizer
        self.device = device
        self.N_trn = len(trainset)
        self.grads_per_elem = None
        self.theta_init = None
        self.num_classes = num_classes
        self.batch_size = batch_size

    def _compute_per_element_grads(self, theta_init):
        self.model.load_state_dict(theta_init)
        batch_wise_indices = np.array(
            [list(BatchSampler(SequentialSampler(self.remainList), self.batch_size, drop_last=False))][0])
        cnt = 0
        for batch_idx in batch_wise_indices:
            inputs = torch.cat(
                [self.trainset[x][0].view(-1, 3, self.trainset[x][0].shape[1], self.trainset[x][0].shape[2]) for x in
                 batch_idx], dim=0).type(torch.float)
            targets = torch.tensor([self.trainset[x][1] for x in batch_idx])
            inputs, targets = inputs.to(self.device), targets.to(self.device, non_blocking=True)
            if cnt == 0:
                with torch.no_grad():
                    data = F.softmax(self.model(inputs), dim=1)
                tmp_tensor = torch.zeros(len(inputs), self.num_classes).to(self.device)
                tmp_tensor.scatter_(1, targets.view(-1, 1), 1)
                outputs = tmp_tensor
                cnt = cnt + 1
            else:
                cnt = cnt + 1
                with torch.no_grad():
                    data = torch.cat((data, F.softmax(self.model(inputs), dim=1)), dim=0)
                tmp_tensor = torch.zeros(len(inputs), self.num_classes).to(self.device)
                tmp_tensor.scatter_(1, targets.view(-1, 1), 1)
                outputs = torch.cat((outputs, tmp_tensor), dim=0)
        grads_vec = data - outputs
        torch.cuda.empty_cache()
        print("Per Element Gradient Computation is Completed")
        self.N_trn = len(grads_vec)
        self.grads_per_elem = grads_vec

    def _update_grads_val(self, theta_init, grads_currX=None, first_init=False):
        self.model.load_state_dict(theta_init)
        self.model.zero_grad()
        if first_init:
            with torch.no_grad():
                scores = F.softmax(self.model(self.x_val), dim=1)
                one_hot_label = torch.zeros(len(self.y_val), self.num_classes).to(self.device)
                one_hot_label.scatter_(1, self.y_val.view(-1, 1), 1)
                grads = scores - one_hot_label
        # populate the gradients in model params based on loss.
        elif grads_currX is not None:
            # update params:
            with torch.no_grad():
                params = [param for param in self.model.parameters()]
                params[-1].data.sub_(self.eta * grads_currX)
                scores = F.softmax(self.model(self.x_val), dim=1)
                one_hot_label = torch.zeros(len(self.y_val), self.num_classes).to(self.device)
                one_hot_label.scatter_(1, self.y_val.view(-1, 1), 1)
                grads = scores - one_hot_label
        self.grads_val_curr = grads.mean(dim=0)  # reset parm.grads to zero!

    def eval_taylor(self, grads_elem, theta_init):
        grads_val = self.grads_val_curr
        dot_prod = 0
        self.model.load_state_dict(theta_init)
        with torch.no_grad():
            params = [param for param in self.model.parameters()]
            dot_prod += torch.sum(grads_val[0] * (params[-1].data - self.eta * grads_elem[0]))
        return dot_prod.data

    def eval_taylor_modular(self, grads, theta_init):
        grads_val = self.grads_val_curr
        self.model.load_state_dict(theta_init)
        with torch.no_grad():
            grads_tensor = torch.cat(grads, dim=0)
            param_update = self.eta * grads_tensor
            gains = torch.matmul(param_update, grads_val)
        return gains

    # Updates gradients of set X + element (basically adding element to X)
    # Note that it modifies the inpute vector! Also grads_X is a list! grad_e is a tuple!
    def _update_gradients_subset(self, grads_X, element):
        grads_e = self.grads_per_elem[element]
        grads_X += grads_e

    # Same as before i.e full batch case! No use of dataloaders here!
    # Everything is abstracted away in eval call
    def naive_greedy_max(self, budget,remainList,theta_init): # REMAIN LIST IS NOT USED!!!!
        self.remainList = remainList
        start_time = time.time()
        self._compute_per_element_grads(theta_init)
        end_time = time.time()
        print("Per Element gradient computation time is: ", end_time - start_time)
        start_time = time.time()
        self._update_grads_val(theta_init, first_init=True)
        end_time = time.time()
        print("Updated validation set gradient computation time is: ", end_time - start_time)
        # Dont need the trainloader here!! Same as full batch version!
        numSelected = 0
        grads_currX = []  # basically stores grads_X for the current greedy set X
        greedySet = list()
        remainSet = list(range(self.N_trn))
        t_ng_start = time.time()  # naive greedy start time
        subset_size = int((len(self.grads_per_elem) / budget) * math.log(100))
        while (numSelected < budget):
            # Try Using a List comprehension here!
            t_one_elem = time.time()
            subset_selected = list(np.random.choice(np.array(list(remainSet)), size=subset_size, replace=False))
            rem_grads = [self.grads_per_elem[x].view(1, self.grads_per_elem[0].shape[0]) for x in subset_selected]
            gains = self.eval_taylor_modular(rem_grads, theta_init)
            # Update the greedy set and remaining set
            bestId = subset_selected[torch.argmax(gains)]
            greedySet.append(bestId)
            remainSet.remove(bestId)
            # Update info in grads_currX using element=bestId
            if numSelected > 0:
                self._update_gradients_subset(grads_currX, bestId)
            else:  # If 1st selection, then just set it to bestId grads
                grads_currX = self.grads_per_elem[bestId]  # Making it a list so that is mutable!
            # Update the grads_val_current using current greedySet grads
            self._update_grads_val(theta_init, grads_currX)
            if numSelected % 1000 == 0:
                # Printing bestGain and Selection time for 1 element.
                print("numSelected:", numSelected, "Time for 1:", time.time() - t_one_elem)
            numSelected += 1
        print("Naive greedy total time:", time.time() - t_ng_start)
        return list(greedySet), grads_currX

In [344]:
from sklearn.model_selection import train_test_split
import copy, datetime, time

def prepare_data(X, y, test_size, val_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = val_size)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [366]:
def greedy_dss(X, y, num_cls, train_batch_size,
               theta, learning_rate, bud, num_epochs, lam, R, sel):
    '''
    Attributes:
    -----
    
    X_train: ndarray
        Training set (U)
    X_val: ndarray
        Validation set (V)
    theta: ndarray
        Vector of current parameters
    learning_rate: int
        Learning rate (eta)
    bud: int
        Budget (k)
    num_epochs: int
        Number of times to do Taylor-series approximation (r)
    lam: int
        Regularization coefficient
    R: function
        Regularization function
    sel: String
        Selection method type, one of {'naive_greedy', 'stochastic_greedy'}
    loss: function
        Function which accepts training subsample and a vector of parameters and returns value of LLT
    loss_grad: function
        Function which accepts training subsample and a vector of parameters and returns gradient of LLT
    '''
    
    # Set train-val-test sets parameters
    device = "cpu"
    
    # Prepare sample
    X_train, y_train, X_val, y_val, X_test, y_test = prepare_data(X, y, 0.3, 0.3)
    
    # Random seeds
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Set up the model
    model = MnistNet() # Choose the model
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    criterion_nored = nn.CrossEntropyLoss(reduction = 'none')
    optimizer = optim.SGD(model.parameters(), lr = learning_rate)
    
    # TODO: Stochastic greedy vs naive greedy?
    idxs = np.arange(X_train.shape[0])
    total_idxs = np.arange(X_train.shape[0])
    
    substrn_losses = np.zeros(num_epochs)
    fulltrn_losses = np.zeros(num_epochs)
    val_losses = np.zeros(num_epochs)
    
    subset_trnloader = torch.utils.data.DataLoader(X_train, batch_size = train_batch_size, 
                                                   shuffle = False,
                                                   sampler = SubsetRandomSampler(idxs),
                                                   pin_memory = True)
    
    setf_model = SetFunctionLoader_2(X_train, X_val, y_val, model, criterion,
                             criterion_nored, learning_rate, device, num_cls, 1000)
    
    for i in range(num_epochs):
        actual_idxs = idxs
        batch_wise_indices = [actual_idxs[x] for x in list(BatchSampler(RandomSampler(actual_idxs), train_batch_size, drop_last=False))]
        subtrn_loss = 0
        for batch_idx in batch_wise_indices:
            inp = np.expand_dims(X[batch_idx, :], 1)
            inp = inp.reshape(len(batch_idx), 1, 8, 8) # HARD_CODED
            inputs = torch.from_numpy(inp).type(torch.float)
            targets = torch.from_numpy(y[batch_idx]).type(torch.LongTensor)
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            subtrn_loss += loss.item()
            loss.backward()
            optimizer.step()

        val_loss = 0
        full_trn_loss = 0

        with torch.no_grad(): # TODO: batches
            inp = np.expand_dims(X_val, 1)
            inp = inp.reshape(X_val.shape[0], 1, 8, 8) # HARD_CODED
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            # TODO: bathces
            inp = np.expand_dims(X_train, 1)
            inp = inp.reshape(X_train.shape[0], 1, 8, 8) # HARD_CODED
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            full_trn_loss += loss.item()

        substrn_losses[i] = subtrn_loss
        fulltrn_losses[i] = full_trn_loss
        val_losses[i] = val_loss
        print('Epoch:', i + 1, 'SubsetTrn,FullTrn,ValLoss:', subtrn_loss, full_trn_loss, val_loss)
        
        cached_state_dict = copy.deepcopy(model.state_dict())
        clone_dict = copy.deepcopy(model.state_dict())
        print("selEpoch: %d, Starting Selection:" % i, str(datetime.datetime.now()))
        subset_idxs, grads_idxs = setf_model.naive_greedy_max(int(bud), clone_dict)
        rem_idxs = list(set(total_idxs).difference(set(subset_idxs)))
        subset_idxs.extend(list(np.random.choice(rem_idxs, size=int((1 - lam) * bud), replace=False)))
        idxs = subset_idxs
        
        print("selEpoch: %d, Selection Ended at:" % (i), str(datetime.datetime.now()))
        model.load_state_dict(cached_state_dict)
        
        ### Change the subset_trnloader according to new found indices: subset_idxs
        subset_trnloader = torch.utils.data.DataLoader(trainset, batch_size=trn_batch_size, shuffle=False,
                                                       sampler=SubsetRandomSampler(idxs), num_workers=1,
                                                       pin_memory=True)
            
    time = start.elapsed_time(end)/1000
    subtrn_loss = 0
    subtrn_correct = 0
    subtrn_total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(subset_trnloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            subtrn_loss += loss.item()
            _, predicted = outputs.max(1)
            subtrn_total += targets.size(0)
            subtrn_correct += predicted.eq(targets).sum().item()
    subtrn_acc = subtrn_correct / subtrn_total

    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
    val_acc = val_correct / val_total

    full_trn_loss = 0
    full_trn_correct = 0
    full_trn_total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            full_trn_loss += loss.item()
            _, predicted = outputs.max(1)
            full_trn_total += targets.size(0)
            full_trn_correct += predicted.eq(targets).sum().item()
    full_trn_acc = full_trn_correct / full_trn_total
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    tst_acc = correct / total

    print("SelectionRun---------------------------------")
    print("Final SubsetTrn and FullTrn Loss:", subtrn_loss, full_trn_loss)
    print("Validation Loss and Accuracy:", val_loss, val_acc)
    print("Test Data Loss and Accuracy:", test_loss, tst_acc)
    print('-----------------------------------')
    return val_acc, tst_acc,  subtrn_acc, full_trn_acc, val_loss, test_loss, subtrn_loss, full_trn_loss, val_losses, substrn_losses, fulltrn_losses, idxs, time

In [367]:
from sklearn.datasets import load_digits

In [368]:
data = load_digits()
X = data['data']
y = data['target']

In [369]:
greedy_dss(X, y, 10, 50, 0, 1, 5, 5, 0.1, None, None)

Epoch: 1 SubsetTrn,FullTrn,ValLoss: 61.68104887008667 2.253173589706421 2.251558303833008
selEpoch: 0, Starting Selection: 2021-03-08 10:33:45.150135


TypeError: naive_greedy_max() missing 1 required positional argument: 'theta_init'